# Part 0:
## import everything
Run the cell below

In [ ]:
import os
import glob
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
import sys
import pickle
import string
import matplotlib as mpl
import matplotlib.pyplot as plt
import PIL
from scipy import stats
from scipy.ndimage.filters import gaussian_filter as smooth
import matplotlib.animation as animation
import matplotlib.backends.backend_pdf
import mpl_toolkits.axes_grid1.inset_locator as inset
from matplotlib.ticker import FormatStrFormatter, MaxNLocator, ScalarFormatter, FuncFormatter
from matplotlib.patches import ConnectionPatch, FancyArrowPatch
from set_rc_params import set_rc_params
import ROOT


if "__file__" not in dir():
    %matplotlib inline
    %config InlineBackend.close_figures = False

    root=ROOT.root
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    CWD=os.getcwd()
    os.chdir(CommonNoteBookesPath)
    %run UtilityTools.ipynb
    %run Animal_Tags.ipynb
    %run loadRat_documentation.ipynb
    %run Lesion_Size.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    %run plotRat_documentation_3_KinematicsInvestigation.ipynb
    %run RunBatchRat_3_CompareGroups.ipynb
    %run BatchRatBehavior.ipynb
    currentNbPath=os.path.join(os.path.split(ThisNoteBookPath)[0],'LesionPaper','LesionSizeLocation.ipynb')
    %run $currentNbPath

    os.chdir(CWD)

    logging.getLogger().setLevel(logging.ERROR)
    
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
    }
    Y1,Y2=param['treadmillRange']

    print('os:',OS(),'\nroot:',root,'\nImport successful!')

---
---


# part 1:

# DEFINITIONS

### If you don't know what to do, move to part 2

In [ ]:
def add_panel_caption(axes: tuple, offsetX: tuple, offsetY: tuple, **kwargs):
    """
    This function adds letter captions (a,b,c,d) to Axes in axes
    at top left, with the specified offset, in RELATIVE figure coordinates
    """
    assert len(axes)==len(offsetX)==len(offsetY), 'Bad input!'
    
    fig=axes[0].get_figure()
    fbox=fig.bbox
    for ax,dx,dy,s in zip(axes,offsetX,offsetY,string.ascii_uppercase):
        axbox=ax.get_window_extent()
    
        ax.text(x=(axbox.x0/fbox.xmax)-abs(dx), y=(axbox.y1/fbox.ymax)+abs(dy),
                s=s,fontweight='extra bold', fontsize=10, ha='left', va='center',
               transform=fig.transFigure,**kwargs)

---

String Format for Scientific Notation

In [ ]:
def SciNote(string):
    """
    Format numbers with Real scientific notation
    Ex: 'p-val={}'.format(SciNote(p))
    """
    f = ScalarFormatter(useOffset=False, useMathText=True)
    g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.2e' % x))
    fmt = FuncFormatter(g)
    return fmt(string)

---

plotting the predefined image

In [ ]:
def plot_animal_image(ax, animal):
    PATHS=('/NAS02/Rat302/Histology/DS_45.jpg',)
    
    try:
        filePath=[path for path in PATHS if animal in path][0]
    except IndexError:
        logging.error(f'Bad Animal name ({animal}), path not defined!')
        return
    
    f=PIL.Image.open(filePath)
    f.thumbnail((500,500),PIL.Image.ANTIALIAS)
    ax.imshow(f)

------



------

# part 2:

# GENERATING THE FIGURE

Definition of Parameters

In [ ]:
if "__file__" not in dir():
    # GENERAL PARAMS
    
    CtrlColor='gray'
    DLSColor='xkcd:red'
    DMSColor='xkcd:blue'
    DSColor='xkcd:magenta'
    
    ColorCode={'DS':DSColor,
               'DMS':DMSColor,
               'DLS':DLSColor,
               'Control':CtrlColor
              }
    
    BadLateRats=('Rat223','Rat231')
       

    
    #===============================================
    
    # GRID 1 PARAMS
       
    
    TaskParamToPlot1="Forward Running Speed"
    
    profilePreLesion1={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout', 'Control-NoTimeout-Control',
                     'Control-Sharp','IncReward-Late-Sharp','Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
              }
    
    profileDLS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DLS']
                 }
    profileDMS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DMS','Late-Lesion_DMS-Sharp'],
                 }  
    profileDS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DS','Late-Lesion_DS-Sharp']
                 }  

    ProfilesDLS1=(profilePreLesion1,profileDLS1,)
    ProfilesDMS1=(profilePreLesion1,profileDMS1,)
    ProfilesDS1 =(profilePreLesion1,profileDS1,)
    
    
    
    #================================================
    
    # GRID 2 PARAMS
    
    sessionBefore2='Rat302_2018_12_12_14_27'
    sessionAfter2 ='Rat302_2019_01_14_14_27'
    
    
    #================================================
    
    # GRID 3 PARAMS

    profileLesions3={'Type':'Good',
                     'rewardType':'Progressive',
                     'option':['not used', 'AsymmetricLesion'],
                     'initialSpeed':['0','10'],
                     'Speed':'10',
                     'Tag': list(set((*profileDLS1['Tag'], *profileDMS1['Tag'], *profileDS1['Tag'])))
                     }

    Profiles3=(profilePreLesion1,profileLesions3)
    
    preSlice3=slice(-5,None)
    postSlice3=slice(0,3)
    finSlice3=slice(8,13)
    
    minSpdReduction3=0
    
    TaskParamToPlot3="Maximum Position"
    
    #================================================
    
    # GRID 4 PARAMS
    
    



Plotting the figure

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc_params({'axes.labelsize':'x-small'})
    figsize=(5,2.5)
    fig=plt.figure(figsize=figsize,dpi=600)
    
        

    ##########################################
    # 1: % Models
#     gs1= fig.add_gridspec(nrows=1, ncols=1, left=0, bottom=0.6, right=1, top=1)
#     ax1= fig.add_subplot(gs1[0])
    
#     ax1.xaxis.set_visible(False)
#     ax1.yaxis.set_visible(False)




    
    
    ##########################################
    # 2: Max Pos EXAMPLE
    gs2= fig.add_gridspec(nrows=2, ncols=1, left=.6, bottom=.6, right=.72, top=1, wspace=0)
#     ax2im= fig.add_subplot(gs2[0])
    ax2L = fig.add_subplot(gs2[0])
    ax2R = fig.add_subplot(gs2[1])

    MeanTrajBefore2=plot_trajectories_and_distributions(root, ax2L, sessionBefore2)
    MeanTrajAfter2 =plot_trajectories_and_distributions(root, ax2R, sessionAfter2)
    

    ax2R.plot(np.linspace(-1,len(MeanTrajBefore2)/25-1,len(MeanTrajBefore2)),MeanTrajBefore2,
              color='navy', lw=2, alpha=.6)

    
    ax2L.set_title('Before (#$-1$)',fontsize='xx-small')
    ax2R.set_title('After (#$+10$)',fontsize='xx-small')
    
    ax2R.tick_params(bottom=True, left=True, labelbottom=True, labelleft=True)
    ax2R.set_xticks([0,1,2,3,4,5,6,7,8,9,10])
    ax2R.set_xticklabels([0,'','','','','','',7,'','',10])
    ax2R.set_yticks([0,45,90])
    ax2R.set_yticklabels([0,'',90])
    ax2R.spines['bottom'].set_bounds(0,10)
    ax2R.set_xlabel('Trial time (s)',labelpad=0,fontsize='xx-small')
    ax2R.set_ylabel('Position (cm)',labelpad=0,fontsize='xx-small')
    ax2R.spines['left'].set_bounds(0,90)   
    ax2R.spines['bottom'].set_visible(True)
    ax2R.spines['left'].set_visible(True)
    ax2R.tick_params('both',labelsize='xx-small')



    ##########################################
    # 3: Normalized speed time course
    gs3= fig.add_gridspec(nrows=1, ncols=1, left=0, bottom=.6, right=0.35, top=1)
    ax3= fig.add_subplot(gs3[0])
    
    
    behav3, AllAnimals3=_late_lesion_effect(root, Profiles=Profiles3,badAnimals=[],TaskParamToPlot=TaskParamToPlot1,
                                            preSlice=preSlice3, postSlice=finSlice3)

    goodAnimals3=[animal for i,animal in enumerate(AllAnimals3) if behav3[i]<minSpdReduction3]
    badAnimals3=list(set(AllAnimals3) - set(goodAnimals3))
    
    data3G,_=plot_normalized_time_course(root, ax3, Profiles3,Animals=goodAnimals3,TaskParamToPlot=TaskParamToPlot1,color='k',shift=-.1*.25)
    data3B,_=plot_normalized_time_course(root, ax3, Profiles3,Animals=badAnimals3,TaskParamToPlot=TaskParamToPlot1,color='gray',shift=.1*.25)

    ax3.text(-5,-25,f'$n={data3G.shape[0]}$ rats',
          ha='left',va='center',fontsize='xx-small',color='k')
    ax3.text(-5,-30,f'$n={data3B.shape[0]}$ rats',
          ha='left',va='center',fontsize='xx-small',color='gray')

    ax3.set_yticks(range(-30,11,10))
    ax3.set_ylabel('Norm. speed (cm/s)',labelpad=0)
    ax3.spines['left'].set_bounds(-30,10)
    ax3.set_ylim([-35,12])

    
    
    
    
    ##########################################
    # 5: Normalized Conditional Max Pos time course
    gs5= fig.add_gridspec(nrows=1, ncols=1, left=.0, bottom=0, right=.35, top=.4)
    ax5= fig.add_subplot(gs5[0])
    
    data5,goodAnimals5=plot_normalized_time_course(root, ax5, Profiles3,Animals=goodAnimals3,
                                                   TaskParamToPlot=TaskParamToPlot3,color='k')
    
    ax5.scatter([-1,10],data5[np.argwhere(goodAnimals5==sessionBefore2[:6]),[4,14]][0],
                c='xkcd:magenta',s=5,marker='v')

    ax5.text(-5,-15,f'$n={data5.shape[0]}$ rats',
          ha='left',va='bottom',fontsize='xx-small',color='k')
    
    ax5.set_ylim([-15,5])
    ax5.set_yticks(range(-15,6,5))
    ax5.set_ylabel('Norm. Max. Pos. (cm)',labelpad=0)
    ax5.spines['left'].set_bounds(-15,5)
    

    
    ##########################################
    # 6: % Lesion size cmparison
    gs6= fig.add_gridspec(nrows=1, ncols=1, left=.4, bottom=0.6, right=0.52, top=1)
    ax6= fig.add_subplot(gs6[0])
    
    
    goodAnimals6=[rat for rat in goodAnimals3 if rat != sessionBefore2[:6]]
    plot_group_lesion_bar(goodAnimals6, ax6, x=1, color='k')
    plot_group_lesion_bar(badAnimals3 , ax6, x=2, color='gray')
    try:
        L6=HistologyExcel('/NAS02',sessionBefore2[:6]).lesion_size()
        ax6.scatter (1.4, L6,
                     s=10, c='xkcd:magenta', marker='v', edgecolors='none', alpha=.8, zorder=2)
        ax6.annotate(s='',xy=(1.4,L6),xytext=(1.6,L6+.2),
                     arrowprops=dict(facecolor='k',arrowstyle='->',shrinkB=2))
    except:
        L6=-1
    
    ax6.set_xlim([.5,2.5])
    ax6.set_xticks([1,2])
    ax6.set_xticklabels(['$\Delta$Speed$<0$','$\Delta$Speed$>0$'], fontsize='xx-small', rotation=15)
    ax6.set_ylim([0,1])
    ax6.set_yticks(np.arange(0,1.01,.1))
    ax6.set_yticklabels([0,'','','','','','','','','',1])
    ax6.set_ylabel('Lesion size',labelpad=-5)
    
    
    
    ##########################################
    # 4: % speed correlation
    gs4= fig.add_gridspec(nrows=1, ncols=1, left=0.48, bottom=0, right=.72, top=.35)
    ax4= fig.add_subplot(gs4[0])
  
    behav4, size4, animals4=late_lesion_correlation_with_size(root, ax=ax4, Profiles=Profiles3, Animals=goodAnimals5,
                                                              color=ColorCode, TaskParamToPlot=TaskParamToPlot3,
                                                              preSlice=preSlice3, postSlice=finSlice3)
    
    _y_=behav4[animals4.index(sessionBefore2[:6])]
    ax4.scatter(L6,_y_,
            c='xkcd:magenta',s=10,marker='v')
    ax4.annotate(s='',xy=(L6,_y_),xytext=(L6-.2,_y_),
                 arrowprops=dict(facecolor='k',arrowstyle='->',shrinkB=2))

    
    r4,p4=stats.pearsonr(size4, behav4)
    s=f'$r=$'+'$%+.2f$'%(r4)+'\n'+'$p=$'+'{}'.format(SciNote(p4))
    ax4.text(1,10,s, ha='right',va='top', fontsize='xx-small')

    ax4.set_ylim([-20,10])
    ax4.spines['left'].set_bounds(-20,10)
    ax4.set_yticks([-20,-10,0,10])
    ax4.set_ylabel('$\Delta$Max. Pos. (cm)')
    
    
    

    
    
    
        
    
    fig.align_ylabels([ax3,ax5])
    ############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    AXES=(ax3,ax2L,ax5,ax4)
    OFFX=np.array([.07]*len(AXES))
    OFFY=np.array([.03]*len(AXES))
    OFFX[[0,2]]=0.05
#     OFFX[[0,1,2,4,6]]=0.05
    
    add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)
    
    fig.savefig(os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','LesionSizeLocation.pdf'),
                format='pdf', bbox_inches='tight')
    
    plt.show()
    plt.close('all')
    matplotlib.rcdefaults()